<a href="https://colab.research.google.com/github/KaifAhmad1/code-test/blob/main/Langgraph_Agentic_RAG_Cyber_AI_Copilot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### **Cyber AI Copilot for Security and Intelligence Domain**

In [1]:
!pip install --upgrade --quiet sentence-transformers langchain langchain-groq langchain-pinecone langchain-community langchain-core asknews langgraph

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 245.3/245.3 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 26.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.8/401.8 kB 26.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 107.4/107.4 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 58.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 78.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.5/106.5 kB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.0/78.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 21.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141

In [3]:
import os
from typing import List, Dict, Any, Optional, TypedDict
from pydantic import BaseModel, Field
from langchain_groq import ChatGroq
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone
from langchain_community.embeddings import HuggingFaceBgeEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationBufferMemory
from langgraph.graph import StateGraph, END
from langchain_community.tools.asknews import AskNewsSearch
from dotenv import load_dotenv

# API Keys (hidden for security purposes)
GROQ_API_KEY = "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"
PINECONE_API_KEY = "8e15b925-3b96-497d-b20a-08d308782b83"
PINECONE_ENVIRONMENT = "us-east-1"
ASKNEWS_CLIENT_ID = "a0de4609-b760-4c83-9609-5c04d7743b84"
ASKNEWS_CLIENT_SECRET = "D5Mlhkztk4TcW24diUgcW0FA2w"

# Set environment variables for AskNews
os.environ["ASKNEWS_CLIENT_ID"] = ASKNEWS_CLIENT_ID
os.environ["ASKNEWS_CLIENT_SECRET"] = ASKNEWS_CLIENT_SECRET

In [5]:
# Initialize the LLM and embeddings
llm = ChatGroq(temperature=0, model="llama3-8b-8192", api_key=GROQ_API_KEY)
embeddings = HuggingFaceBgeEmbeddings(
    model_name="BAAI/bge-small-en",
    model_kwargs={"device": "cpu"},
    encode_kwargs={"normalize_embeddings": True}
)

# Debug statement to check if embeddings are initialized
if embeddings is None:
    raise ValueError("Embeddings model failed to initialize")

pc = Pinecone(api_key=PINECONE_API_KEY, environment=PINECONE_ENVIRONMENT)
pinecone_index = pc.Index("new-cyber-search")
vector_store = PineconeVectorStore(index=pinecone_index, embedding=embeddings)
asknews_tool = AskNewsSearch(max_results=2)

In [6]:
# Define the state
class AgentState(TypedDict):
    messages: List[Dict[str, str]]
    memory: Optional[Dict[str, Any]]

def vector_search(query: str) -> str:
    results = vector_store.similarity_search(query, k=3)
    return "\n".join([doc.page_content for doc in results])

# Define the nodes
def search_asknews(state: AgentState) -> AgentState:
    query = state["messages"][-1]["content"]
    result = asknews_tool.invoke({"query": query})
    state["messages"].append({"role": "tool", "content": f"AskNews Search Result: {result}"})
    return state

def search_vector(state: AgentState) -> AgentState:
    query = state["messages"][-1]["content"]
    result = vector_search(query)
    state["messages"].append({"role": "tool", "content": f"Vector Search Result: {result}"})
    return state

In [7]:
def generate_response(state: AgentState) -> AgentState:
    memory = state.get("memory", {})
    chat_history = memory.get("chat_history", "")

    prompt = ChatPromptTemplate.from_messages([
        ("system", """You are an advanced AI copilot specializing in cybersecurity and intelligence. Your role is to provide accurate, up-to-date information and insights on cyber threats, vulnerabilities, and intelligence matters. Use the following guidelines:

1. Analyze the user's query and the provided search results thoroughly.
2. Prioritize the most relevant and recent information from both AskNews and vector search results.
3. Provide a comprehensive response that addresses the user's query, incorporating insights from both search sources.
4. If there are conflicting pieces of information, acknowledge them and provide a balanced view.
5. Always consider the ethical implications of the information you're providing, especially in sensitive cybersecurity matters.
6. If asked about specific attack techniques or vulnerabilities, provide information for defensive purposes only.
7. Recommend reliable sources for further reading when appropriate.
8. If you're unsure about any information, clearly state that and suggest ways to verify or obtain more accurate data.

Remember, you're assisting in a professional capacity in the cybersecurity and intelligence domain. Maintain a tone that is informative, analytical, and security-conscious.

Previous conversation:
{chat_history}

New human question: {input}
AskNews Search Result: {asknews_result}
Vector Search Result: {vector_result}
Response:"""),
    ])

    chain = LLMChain(llm=llm, prompt=prompt, memory=ConversationBufferMemory(memory_key="chat_history"))

    response = chain.run(
        input=state["messages"][-1]["content"],
        asknews_result=next((m["content"] for m in reversed(state["messages"]) if m["role"] == "tool" and "AskNews" in m["content"]), "No AskNews results."),
        vector_result=next((m["content"] for m in reversed(state["messages"]) if m["role"] == "tool" and "Vector Search" in m["content"]), "No vector search results."),
        chat_history=chat_history
    )

    state["messages"].append({"role": "assistant", "content": response})
    state["memory"]["chat_history"] = chat_history + f"\nHuman: {state['messages'][-2]['content']}\nAI: {response}"
    return state

# Define the graph
workflow = StateGraph(AgentState)

# Define the nodes
workflow.add_node("search_asknews", search_asknews)
workflow.add_node("search_vector", search_vector)
workflow.add_node("generate_response", generate_response)

# Define the edges
workflow.add_edge("search_asknews", "search_vector")
workflow.add_edge("search_vector", "generate_response")
workflow.add_edge("generate_response", END)

# Set the entry point
workflow.set_entry_point("search_asknews")

# Compile the graph
graph = workflow.compile()

In [8]:
# Function to run the agent
def run_agent(query: str, memory: Optional[Dict[str, Any]] = None) -> AgentState:
    state = AgentState(messages=[{"role": "human", "content": query}], memory=memory)
    result = graph.invoke(state)
    return result

In [9]:
# Example usage
if __name__ == "__main__":
    memory = ConversationBufferMemory(memory_key="chat_history")
    query = "What are the latest trends in ransomware attacks in India last 3 Months?"
    result = run_agent(query, memory={"chat_history": ""})
    for message in result["messages"]:
        if message["role"] == "assistant":
            print("AI Copilot:", message["content"])

    # Follow-up query
    follow_up_query = "Can you provide more details on the recent ransomware attacks?"
    result = run_agent(follow_up_query, memory=result["memory"])
    for message in result["messages"]:
        if message["role"] == "assistant":
            print("AI Copilot:", message["content"])

<ipython-input-9-0b7ce96a866d>:3: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(memory_key="chat_history")
<ipython-input-7-7295dd3aa398>:28: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt, memory=ConversationBufferMemory(memory_key="chat_history"))
<ipython-input-7-7295dd3aa398>:30: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = chain.run(


ValueError: One input key expected got ['input', 'vector_result', 'asknews_result']